<h2> Capstone Project: Battle of the Neighborhoods

In [18]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [19]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [20]:
!pip install geopy

You should consider upgrading via the 'c:\users\samete\anaconda3\python.exe -m pip install --upgrade pip' command.


In [21]:
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from sklearn.cluster import KMeans
import folium # map rendering library

In [22]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.request('GET', URL)

**Creating the soup object for the data analysis :**

In [23]:
soup = BeautifulSoup(page.content, "html.parser")

**Finding the tables within the HTML code, and assigning them to a table object :**

In [24]:
table = soup.find(name="table")

In [25]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

**Checking some elements of the list "table_contents"**

In [26]:
table_contents[:4]

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'},
 {'PostalCode': 'M4A',
  'Borough': 'North York',
  'Neighborhood': 'Victoria Village'},
 {'PostalCode': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Regent Park, Harbourfront'},
 {'PostalCode': 'M6A',
  'Borough': 'North York',
  'Neighborhood': 'Lawrence Manor, Lawrence Heights'}]

In [27]:
df = pd.DataFrame(data = table_contents)

In [28]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


<h2>First part of the assignment :

In [29]:
df.shape

(103, 3)

**Now for getting the geolocations (unfortunately geocoder did not work for me)**

**Since geocoder did not work, I'm going to use the csv data provided**

In [34]:
lat_lng_coords = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")

In [35]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [36]:
lat_lng_coords.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


**Joining the base table(df) with the coordinates table (lat_long_coords):**

In [37]:
neighborhoods = df.join(lat_lng_coords, how='left')
neighborhoods.drop(columns=['Postal Code'], inplace=True)

<h2>Second part of the assignment :

In [39]:
neighborhoods.head(5)
len(neighborhoods)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Ontario Provincial Government,43.773136,-79.239476


103

In [40]:
print('The neighborhood data consists of {} rows, {} unique Boroughs'
      .format(neighborhoods.shape[0],neighborhoods.Borough.nunique()))

The neighborhood data consists of 103 rows, 15 unique Boroughs


<h1> From this point on, I will be working on the third part of the assignment

**Let's get the coordinates of Toronto first**

In [41]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


**For each Borough in the neighborhoods dataframe, we are creating a popup point on the map with relevant labels:**

In [42]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

**Aaaand here is the map with coordinate points:**

In [43]:
map_toronto

**Getting the Boroughs containing "Toronto" in their names:**

In [44]:
nbr_toronto = neighborhoods[neighborhoods.Borough.str.contains('Toronto')].reset_index(drop=True)

# create map of New York using latitude and longitude values
map_toronto_only = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nbr_toronto['Latitude'], nbr_toronto['Longitude'], nbr_toronto['Borough'], nbr_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_only)  

**Neighborhoods containing "Toronto" only:**

In [45]:
map_toronto_only

In [46]:
CLIENT_ID = 'LKPF30LKMYSAKW0PKO5AQSQ1QOXOQ3FDJFMR1ZYWBCKRST0A' # your Foursquare ID
CLIENT_SECRET = 'E3DJ5ZKXPUTXURURPZSQ2DQRYFQBIHGGJTYHCPPMHK3P3OI3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LKPF30LKMYSAKW0PKO5AQSQ1QOXOQ3FDJFMR1ZYWBCKRST0A
CLIENT_SECRET:E3DJ5ZKXPUTXURURPZSQ2DQRYFQBIHGGJTYHCPPMHK3P3OI3


<i> From now on, we will use "nbr_toronto" as data, "map_toronto_only" as our map

In [53]:
nbr_toronto.head(5)
nbr_toronto.shape

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
3,M4E,East Toronto,The Beaches,43.786947,-79.385975
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714


(39, 5)

In [54]:
neighborhood_latitude = nbr_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = nbr_toronto.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = nbr_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.7635726, -79.1887115.


In [55]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LKPF30LKMYSAKW0PKO5AQSQ1QOXOQ3FDJFMR1ZYWBCKRST0A&client_secret=E3DJ5ZKXPUTXURURPZSQ2DQRYFQBIHGGJTYHCPPMHK3P3OI3&v=20180605&ll=43.7635726,-79.1887115&radius=500&limit=100'

In [56]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60fe5c01dfdf6825eb14c01b'},
 'response': {'headerLocation': 'Scarborough Village',
  'headerFullLocation': 'Scarborough Village, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.768072604500006,
    'lng': -79.18249216787879},
   'sw': {'lat': 43.7590725955, 'lng': -79.1949308321212}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4beee041e24d20a1cd857314',
       'name': 'RBC Royal Bank',
       'location': {'address': '4374 KINGSTON RD',
        'crossStreet': 'Kingston & Lawrence',
        'lat': 43.76678992471017,
        'lng': -79.19115118872593,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.76678992471017,
          'lng': -79.1911511887

In [64]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [65]:
from pandas.io.json import json_normalize

In [59]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-59-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.766790,-79.191151
1,G & G Electronics,Electronics Store,43.765309,-79.191537
2,Sail Sushi,Restaurant,43.765951,-79.191275
3,Big Bite Burrito,Mexican Restaurant,43.766299,-79.190720
4,Enterprise Rent-A-Car,Rental Car Location,43.764076,-79.193406


**Creating the function for repeating the process, this time for all the neighborhoods:**

In [66]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
toronto_venues = getNearbyVenues(names=nbr_toronto['Neighborhood'],
                                   latitudes=nbr_toronto['Latitude'],
                                   longitudes=nbr_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

**EDA of the data created:**

In [77]:
toronto_venues.head()
print(toronto_venues.shape)
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


(758, 7)
There are 204 uniques categories.


**One hot encoding:**

In [105]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
##fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
##toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot = toronto_onehot.reindex(columns=(
    ['Neighborhood'] + list([a for a in toronto_onehot.columns if a != 'Neighborhood'])))

toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health Food Store,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [106]:
toronto_onehot.shape

(758, 204)

In [107]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health Food Store,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.0000,0.000000,0.055556,0.027778,0.027778,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.055556,0.00000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.055556,0.0000,0.000000,0.027778,0.000000,0.000000,0.027778,0.000000,0.083333,0.00,0.0,0.027778,0.0000,0.000000,0.000000,0.027778,0.000000,0.027778,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0000,0.0000,0.000000,0.055556,0.027778,0.0,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.00000

In [108]:
toronto_grouped.shape

(38, 204)

**Let's print each neighborhood along with the top 5 most common venues**

In [109]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0          Coffee Shop  0.08
1              Brewery  0.06
2               Bakery  0.06
3  American Restaurant  0.06
4            Gastropub  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0    Light Rail Station  0.12
1  Gym / Fitness Center  0.06
2                Garden  0.06
3  Fast Food Restaurant  0.06
4            Skate Park  0.06


----Central Bay Street----
            venue  freq
0        Pharmacy  0.14
1  Discount Store  0.14
2   Grocery Store  0.14
3     Pizza Place  0.14
4     Coffee Shop  0.14


----Christie----
                             venue  freq
0             Fast Food Restaurant  0.33
1                             Park  0.33
2                Food & Drink Shop  0.33
3  Molecular Gastronomy Restaurant  0.00
4                     Liquor Store  0.00


----Church and Wellesley----
       

**Let's put that into a dataframe**

In [140]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [141]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Brewery,Bakery,American Restaurant,Gastropub,Café,Cheese Shop,Bookstore,Middle Eastern Restaurant,Fish Market
1,"CN Tower, King and Spadina, Railway Lands, Har...",Light Rail Station,Gym / Fitness Center,Garden,Fast Food Restaurant,Skate Park,Park,Spa,Burrito Place,Restaurant,Farmers Market
2,Central Bay Street,Pharmacy,Discount Store,Grocery Store,Pizza Place,Coffee Shop,Supermarket,Butcher,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
3,Christie,Fast Food Restaurant,Park,Food & Drink Shop,Molecular Gastronomy Restaurant,Liquor Store,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
4,Church and Wellesley,Discount Store,Pizza Place,Playground,Middle Eastern Restaurant,Coffee Shop,Chinese Restaurant,Sandwich Place,Intersection,Airport,Mexican Restaurant


## Cluster Neighborhoods

In [142]:
kclusters = 5

In [143]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

In [144]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [145]:
kmeans.labels_

array([0, 0, 0, 3, 0, 1, 0, 0, 0, 4, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [146]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = nbr_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [147]:
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,0.0,Restaurant,Rental Car Location,Mexican Restaurant,Electronics Store,Donut Shop,Intersection,Bank,Medical Center,Music Venue,Office
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,0.0,Farm,Café,Skating Rink,General Entertainment,College Stadium,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,0.0,Pizza Place,Indian Restaurant,Pharmacy,Burger Joint,Sandwich Place,Bank,Breakfast Spot,Chinese Restaurant,Discount Store,Fast Food Restaurant
3,M4E,East Toronto,The Beaches,43.786947,-79.385975,0.0,Café,Japanese Restaurant,Chinese Restaurant,Bank,Airport,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259,0.0,Pharmacy,Discount Store,Grocery Store,Pizza Place,Coffee Shop,Supermarket,Butcher,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
6,M6G,Downtown Toronto,Christie,43.753259,-79.329656,3.0,Fast Food Restaurant,Park,Food & Drink Shop,Molecular Gastronomy Restaurant,Liquor Store,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.737473,-79.464763,0.0,Airport,Park,Business Service,Monument / Landmark,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.739015,-79.506944,0.0,Park,Grocery Store,Shopping Mall,Bank,Airport,Molecular Gastronomy Restaurant,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant
9,M4J,East YorkEast Toronto,The Danforth East,43.706397,-79.309937,0.0,Pizza Place,Pharmacy,Athletics & Sports,Flea Market,Café,Intersection,Bank,Breakfast Spot,Gastropub,Gym / Fitness Center


## Creating the final map

In [148]:
toronto_merged.drop(axis=0, index=4, inplace=True) #deleting the "M5E" where no nearby venues was returned
toronto_merged.reset_index(inplace=True)

In [158]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int32')

In [159]:
toronto_merged

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,0,Restaurant,Rental Car Location,Mexican Restaurant,Electronics Store,Donut Shop,Intersection,Bank,Medical Center,Music Venue,Office
1,1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,0,Farm,Café,Skating Rink,General Entertainment,College Stadium,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
2,2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,0,Pizza Place,Indian Restaurant,Pharmacy,Burger Joint,Sandwich Place,Bank,Breakfast Spot,Chinese Restaurant,Discount Store,Fast Food Restaurant
3,3,M4E,East Toronto,The Beaches,43.786947,-79.385975,0,Café,Japanese Restaurant,Chinese Restaurant,Bank,Airport,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
4,5,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259,0,Pharmacy,Discount Store,Grocery Store,Pizza Place,Coffee Shop,Supermarket,Butcher,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
5,6,M6G,Downtown Toronto,Christie,43.753259,-79.329656,3,Fast Food Restaurant,Park,Food & Drink Shop,Molecular Gastronomy Restaurant,Liquor Store,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
6,7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.737473,-79.464763,0,Airport,Park,Business Service,Monument / Landmark,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
7,8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.739015,-79.506944,0,Park,Grocery Store,Shopping Mall,Bank,Airport,Molecular Gastronomy Restaurant,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant
8,9,M4J,East YorkEast Toronto,The Danforth East,43.706397,-79.309937,0,Pizza Place,Pharmacy,Athletics & Sports,Flea Market,Café,Intersection,Bank,Breakfast Spot,Gastropub,Gym / Fitness Center
9,10,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.695344,-79.318389,0,Skating Rink,Park,Beer Store,Curling Ice,Dance Studio,Athletics & Sports,Airport,Monument / Landmark,Mediterranean Restaurant,Mexican Restaurant


In [160]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [162]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    

In [163]:
map_clusters

## Naming clusters based on shared characteristics

In [171]:
toronto_merged[toronto_merged['Cluster Labels'] == 0] ## Cluster name : The commons

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,0,Restaurant,Rental Car Location,Mexican Restaurant,Electronics Store,Donut Shop,Intersection,Bank,Medical Center,Music Venue,Office
1,1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,0,Farm,Café,Skating Rink,General Entertainment,College Stadium,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
2,2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,0,Pizza Place,Indian Restaurant,Pharmacy,Burger Joint,Sandwich Place,Bank,Breakfast Spot,Chinese Restaurant,Discount Store,Fast Food Restaurant
3,3,M4E,East Toronto,The Beaches,43.786947,-79.385975,0,Café,Japanese Restaurant,Chinese Restaurant,Bank,Airport,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
4,5,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259,0,Pharmacy,Discount Store,Grocery Store,Pizza Place,Coffee Shop,Supermarket,Butcher,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
6,7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.737473,-79.464763,0,Airport,Park,Business Service,Monument / Landmark,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
7,8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.739015,-79.506944,0,Park,Grocery Store,Shopping Mall,Bank,Airport,Molecular Gastronomy Restaurant,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant
8,9,M4J,East YorkEast Toronto,The Danforth East,43.706397,-79.309937,0,Pizza Place,Pharmacy,Athletics & Sports,Flea Market,Café,Intersection,Bank,Breakfast Spot,Gastropub,Gym / Fitness Center
9,10,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.695344,-79.318389,0,Skating Rink,Park,Beer Store,Curling Ice,Dance Studio,Athletics & Sports,Airport,Monument / Landmark,Mediterranean Restaurant,Mexican Restaurant
10,11,M6J,West Toronto,"Little Portugal, Trinity",43.676357,-79.293031,0,Health Food Store,Trail,Pub,Airport,Monument / Landmark,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant


In [172]:
toronto_merged[toronto_merged['Cluster Labels'] == 1] ## Cluster name : "Casual and better living"

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,16,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.689574,-79.38316,1,Park,Summer Camp,Airport,Molecular Gastronomy Restaurant,Liquor Store,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant


In [173]:
toronto_merged[toronto_merged['Cluster Labels'] == 2] ## Cluster name : "Fine dining, i mean living"

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,33,M4W,Downtown Toronto,Rosedale,43.636258,-79.498509,2,Baseball Field,Airport,Light Rail Station,Liquor Store,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
35,36,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.724766,-79.532242,2,Baseball Field,Furniture / Home Store,Airport,Monument / Landmark,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


In [174]:
toronto_merged[toronto_merged['Cluster Labels'] == 3] ## Cluster name : "Passer thru"

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,6,M6G,Downtown Toronto,Christie,43.753259,-79.329656,3,Fast Food Restaurant,Park,Food & Drink Shop,Molecular Gastronomy Restaurant,Liquor Store,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant


In [175]:
toronto_merged[toronto_merged['Cluster Labels'] == 4] ## Cluster name : "Italian neighborhood?"

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,28,M6S,West Toronto,"Runnymede, Swansea",43.673185,-79.487262,4,Pizza Place,Convenience Store,Brewery,Bus Line,Moroccan Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
37,38,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.688905,-79.554724,4,Pizza Place,Mobile Phone Shop,Bus Line,Sandwich Place,Pharmacy,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant
